## 安装

需要安装一些库

In [1]:
import networkx as nx
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://mirrors.aliyun.com/pypi/simple/, https://pypi.doubanio.com/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://mirrors.aliyun.com/pypi/simple/, https://pypi.doubanio.com/simple


In [2]:
# 加载chatGLM模型并测试一下是否加载成功,让模型写一份信
# 出错的话请自己配置好chatGLM需要的环境,并且检查CUDA是否出错

# chatGLM的模型地址是https://huggingface.co/THUDM/chatglm-6b-int4,任意目录下git clone即可
# chatGLM的github地址是：https://github.com/THUDM/ChatGLM-6B,加载模型出错请自行查询chatGLM加载出错的信息

from transformers import AutoModel, AutoTokenizer
# 这里加载了chatGLM-6b-int4模型,请更换路径到你保存模型的路径
model_path = "C:\AIproject\ChatGLM-6B\model\chatglm-6b-int4"
# 也可以选择从huggingface下载
# model_path = "THUDM/chatglm-6b-int4"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
model = model.eval()
prompt = "写一份关于感恩的信"

def chatGLM_generate(prompt,max_length):
    input = prompt
    for response, chat_history in model.stream_chat(tokenizer, input, history=None,max_length=max_length, top_p=0.7, temperature=1):
        response = chat_history[-1]
    return response

out_text=chatGLM_generate(prompt, max_length=128)

print(out_text)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : C:\Users\Madaramebaku\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\Madaramebaku\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.c -shared -o C:\Users\Madaramebaku\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.so
Load parallel cpu kernel failed, using default cpu kernel code:
Traceback (most recent call last):
  File "C:\Users\Madaramebaku/.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization.py", line 145, in __init__
    kernels = ctypes.cdll.LoadLibrary(kernel_file)
  File "c:\Users\Madaramebaku\AppData\Local\Programs\Python\Python310\lib\ctypes\__init__.py", line 452, in LoadLibrary
    return self._dlltype(name)
  File "c:\Users\Madaramebaku\AppData\Local\Programs\Python\Python310\lib\ctypes\__init__.py", line 37

In [5]:
import networkx as nx
prompt = "谁是第一个登上月球的人？"
def generate(generate_prompt):
  out = chatGLM_generate(prompt=generate_prompt,max_length=len(generate_prompt)+128)
  return out[1]

print(generate(prompt))

第一个登上月球的人是美国宇航员尼尔·阿姆斯特朗(Neil Armstrong)。1969年7月20日,他成为了人类历史上第一个登上月球的人。


## 世界描述

我们描述下面的世界。我们将根据这些信息生成提示。模拟的是无冬城西南的芬达林镇。之所以选择这个区域,是因为它很容易扩展成多个区域,让“玩家”能够在模拟完成后探索世界。

In [5]:
world_graph = nx.Graph()

prompt_meta = "{}"

town_areas = ["巴瑟恩杂货店", "狮盾贸易公司", "斯通希尔旅店", "芬达林镇广场"]
town_areas = {
"芬达林镇广场": "芬达林镇的市中心广场。",
"石山客栈": "镇中心有一座由石块和粗糙板材建成的大型路边店。公共休息室里坐着当地人,他们抱着麦酒或苹果酒,都好奇地盯着你看。",
"巴瑟恩的货栈": "芬达林最大的贸易站是巴瑟恩的货栈。货架上摆满了大部分普通物品和用品,包括背包、棉被、绳索和口粮。这个地方从日出到日落都开放。",
"艾德玛斯果园": "一个整洁的小木屋旁边的苹果园。",
"狮盾集团": "这家不起眼的交易站前门上悬挂着一个木制盾牌形状的招牌,上面画着一头蓝色的狮子。这座建筑由狮盾公司所拥有,该公司总部位于东面100多英里的亚塔市。他们向芬达林和其他小定居点运送成品,但这个前哨站受到了强盗的严重打击。最近应该会到达芬达林的狮盾商队没有出现。",
"芬达林矿工交易所": "矿工交易所是当地矿工把他们有价值的发现放在那里称重、测量并支付报酬的贸易站。在缺乏任何本地领主或权威的情况下,该交易所还充当非官方的记录办公室,在该区域注册各种流和开挖权利。芬达林没有真正的淘金热,但足够的财富隐藏在附近的小溪和山谷中,可以支持相当数量的独立探矿者。该交易所是一个很好的地方,可以结识到经常在芬达林周围的乡村活动的人。会长是一位名叫哈利亚·桑德尔顿的野心勃勃、精明计算的人类女性。",
"奥德利芙农场": "由乐于助人的半身人农民奎琳·奥德利芙拥有的农场。",
"幸运神殿": "芬达林唯一的寺庙,这是一个由附近废墟中取来的石头建成的小神殿。它致力于侍奉提摩拉女神——幸运和好运女神。",
"沉睡巨人": "这个破旧的酒馆位于芬达林主街尽头,是一个肮脏、危险的水源。红袍暴徒经常光顾,由妮莉·星匠经营。",
"镇长大厅": "镇长大厅有坚固的石墙、倾斜的木屋顶和后面的钟楼。在前门旁边的一块板子上张贴着一则用通用语写成的告示。上面写着：“悬赏通告——温沃恩山区附近有兽人！那些有勇气直面兽人威胁的人应该来这里打听打听。”这则告示上印有小镇的印章和无法辨认的签名。",
"特雷森达庄园": "红袍暴徒团伙在芬达林的基地是位于特雷森达庄园的地下室。在这个庄园被废弃之前,它的地下室被用作存储食物和水的安全场所以备战不测,同时相邻的墓穴为特雷森达家族已故成员提供了一个安息之所。自从红袍暴徒团伙接管后,他们已经扩建了地下室,加入了奴隶囚笼、车间和营房等设施来满足他们自己的目的。"
}
town_people = {
"托布伦·斯通希尔": "托布伦拥有一家贸易站。",
"达兰·艾德玛斯": "达兰是一位退役冒险家,住在一个整洁的小木屋旁边的苹果园里。他是一位身材健壮、银发半精灵,已经过百岁的战士,在东南方的龙海岸领地中担任元帅和传令官多年。退役后,他回到了他的故乡——尼瓦伦地区。",
"莱宁·格雷温德": "莱宁经营着一家贸易站。",
"哈莉亚·桑顿": "哈莉亚是一个野心勃勃、精明计算的人类女性。她是芬达林矿工交易所的行会大师,该贸易站称重、测量并支付当地矿工有价值的发现。为了将矿工交易所建立成城镇最接近治理权力的事物,她不止是一个简单的商人。",
"奎琳·奥德利芙": "奎琳是一位45岁充满智慧的矮人农民,似乎知道镇上发生的一切。她是友善的人,如果他们不想住在巴瑟恩的货栈,愿意让流浪者留在她的干草棚里。",
"盖蕾尔修女": "盖蕾尔修女是提摩拉的精灵神职人员。",
"哈宾·韦斯特": "哈宾是芬达林镇长。一个自命不凡、年迈的食品商。芬达林没有政府,但镇民每年选出一个人来担任镇长。镇长在一些小的纠纷中担任法官,并保管需要保管的任何记录。",
"泰瑞尔·血痕": "泰瑞尔是一个人类恶棍。他穿着肮脏的绯红色斗篷,并且是红袍暴徒的成员。他不喜欢冒险家,想抢劫和杀死冒险家。",
"妮莉·星匠": "妮莉是一个人类恶霸。她穿着肮脏的绯红色斗篷,并且是红袍暴徒的成员。她不喜欢冒险家,想抢劫和杀死冒险家。",
}
for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "芬达林镇广场")
locations = {}
for i in town_people.keys():
  locations[i] = "芬达林镇广场"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "现在是 "+str(global_time)+":00. 地点是 "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)


In [6]:
compressed_memories_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = []

In [7]:
for name in town_people.keys():
  prompt = "你是 {}。{} 你刚在芬达林镇广场醒来然后去了镇广场。下面的人住在镇上: {}。你今天的目标是什么？从你的角度回答,最多用30个字来描述".format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

-the prompt:
今天我的目标是探索镇子并了解每个人的故事。
-the out
托布伦·斯通希尔 今天我的目标是探索镇子并了解每个人的故事。
-the prompt:
目标：保护苹果园,迎接新挑战。
-the out
达兰·艾德玛斯 目标：保护苹果园,迎接新挑战。
-the prompt:
我需要与镇上的人交流并了解他们的需要和目标,以便为他们提供最好的服务。
-the out
莱宁·格雷温德 我需要与镇上的人交流并了解他们的需要和目标,以便为他们提供最好的服务。
-the prompt:
哈莉亚的目标是建立矿工交易所成为城镇最接近治理权力的事物。
-the out
哈莉亚·桑顿 哈莉亚的目标是建立矿工交易所成为城镇最接近治理权力的事物。
-the prompt:
探索芬达林镇,了解镇上的人和事情。
-the out
奎琳·奥德利芙 探索芬达林镇,了解镇上的人和事情。
-the prompt:
我需要和所有人交流并了解他们的目标。
-the out
盖蕾尔修女 我需要和所有人交流并了解他们的目标。
-the prompt:
查看芬达林镇的未来
-the out
哈宾·韦斯特 查看芬达林镇的未来
-the prompt:
杀死所有人。
-the out
泰瑞尔·血痕 杀死所有人。
-the prompt:
今天我的目标就是杀死所有红袍暴徒,让他们知道冒险家的滋味。
-the out
妮莉·星匠 今天我的目标就是杀死所有红袍暴徒,让他们知道冒险家的滋味。


In [8]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    prompt = "你是 {}. {} 你计划去做: {}. 你目前在 {} 其描述如下: {}. 当前时间为 {}:00. 这个区域里有以下人员: {}. 你可以其中某些人进行互动。".format(name, town_people[name], plans[name], location, town_areas[location], str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i+': '+town_people[i])
    prompt += ' 你知道以下人员的信息: ' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "接下来的一个小时你要做什么?最多用20个字来描述。"
    action_prompts[name] = prompt

In [9]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  # Now clean the action
  prompt = """
  将以下段落转换为第一人称过去式:
  "{}"
  """.format(action_results[name])
  action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
  print(name, action_results[name])

-the prompt:
与达兰交流,了解其退役后的冒险经历,与哈莉亚探讨矿工交易所,与奎琳交流矮人农业知识,与盖蕾尔修女了解精灵神职人员的生活,与哈宾讨论镇上发生的事件,与泰瑞尔交流红袍暴徒的过去和现在。
-the out
-the prompt:
与达兰交流,了解其退役后的冒险经历。与哈莉亚探讨矿工交易所。与奎琳交流矮人农业知识。与盖蕾尔修女了解
-the out
托布伦·斯通希尔 与达兰交流,了解其退役后的冒险经历。与哈莉亚探讨矿工交易所。与奎琳交流矮人农业知识。与盖蕾尔修女了解
-the prompt:
保护苹果园,与村民互动
-the out
-the prompt:
我保护了苹果园并和村民互动。
-the out
达兰·艾德玛斯 我保护了苹果园并和村民互动。
-the prompt:
与镇上的人交流,了解他们的需求和目标,为他们提供最好的服务。
-the out
-the prompt:
"我曾经与镇上的人交流,了解他们的需求和目标,并为他们提供最好的服务。"
-the out
莱宁·格雷温德 我曾经与镇上的人交流,了解他们的需求和目标,并为他们提供最好的服务。
-the prompt:
与哈莉亚交流,建立矿工交易所成为城镇最接近治理权力的事物。
-the out
-the prompt:
与哈莉亚交流,建立矿工交易所成为城镇最接近治理权力的事物。
-the out
哈莉亚·桑顿 与哈莉亚交流,建立矿工交易所成为城镇最接近治理权力的事物。
-the prompt:
与哈莉亚交流
-the out
-the prompt:
与哈莉亚交流
-the out
奎琳·奥德利芙 与哈莉亚交流
-the prompt:
与所有人交流,了解他们的目标,了解追随者的情况。
-the out
-the prompt:
我与交流所有人交流,了解他们的目标,并了解追随者的情况。
-the out
盖蕾尔修女 我与交流所有人交流,了解他们的目标,并了解追随者的情况。
-the prompt:
查看镇的未来
-the out
-the prompt:
我曾经查看过镇的未来。
-the out
哈宾·韦斯特 我曾经查看过镇的未来。
-the prompt:
杀死所有人
-the out
-the prompt:
我杀死了所有人。
-the out
泰瑞尔·血痕 我杀死了所有人。
-

收集人们观察到的记忆。

In [12]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    for name_two in people:
      memories[name].append('[时间: {}. 人物: {}. 记忆: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

# 记忆排序

In [13]:
import re
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [15]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    prompt = "你是 {}. 你计划: {}. 你目前在 {}. 当前时间为 {}:00. 你可以观察到以下内容: {}. 将你对于这件事关心的程度评为1到5分。".format(name, plans[name], locations[name], str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((res, rating))
  print(memory_ratings[name])

-the prompt:
托布伦·斯通希尔对于探索芬达林镇子并了解每个人的故事非常关心。他正在认真倾听达兰、哈莉亚、奎琳和盖蕾尔修女的故事,并试图了解他们的观点和经历。
-the out
-the prompt:
1.5分。
-the out
-the prompt:
作为托布伦·斯通希尔,我对芬达林镇广场的每个人的故事都很感兴趣。当前时间为8:00,我会花一些时间与莱宁·格雷温德交流,了解他的需求和目标,并尽力为他提供最好的服务。我将把这个过程视为非常关心,评分为5分。
-the out
-the prompt:
我对于这件事的关心程度为3分。我在芬达林镇广场,观察到哈莉亚·桑顿,并与她交流建立矿工交易所成为城镇最接近治理权力的事物。虽然这听起来是一个重要的计划,但是我对矿工交易所的建立并没有太大的兴趣,因为我更喜欢探索每个人的故事。
-the out
-the prompt:
根据观察,我对奎琳·奥德利芙和哈莉亚的故事感兴趣,因此将我对她的兴趣程度评为5分。
-the out
-the prompt:
根据所提供的信息,我认为我对于探索芬达林镇并了解每个人的故事并不太关心。我的主要目标是与盖蕾尔修女交流,了解她的目标和追随者的情况。然而,我并没有提到我对于其他人物的故事有任何的兴趣。因此,我对于这件事的关心程度评为1分。
-the out
-the prompt:
根据提供的信息,我认为我对于探索芬达林镇并了解每个人的故事并没有太大的关心程度。
-the out
-the prompt:
我对于这件事的关心程度为1分。当前时间位于芬达林镇广场,我看到泰瑞尔·血痕,但他似乎与一个记忆有关,提到了杀死所有人。虽然这个事件可能对于镇子的居民构成威胁,但我认为这并不应该影响
-the out
-the prompt:
我对于妮莉·星匠与红袍暴徒战斗的故事非常关心。我的目标是探索芬达林镇并了解每个人的故事,因此我对这个任务非常感兴趣。我将评分为4分,因为我真的很想更多地了解这个镇子
-the out
-the prompt:
根据所提供的信息,我认为我对于探索芬达林镇并了解每个人的故事的关心程度为4分。我与达兰的交流让我更加深入地了解了他退役后的冒险经历,与哈莉亚探讨矿工交易所让我学到了新的知识,与奎琳交流矮
-the out
-the prompt:
我对于这件事关心的

# 压缩记忆

In [17]:
MEMORY_LIMIT = 10
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name], 
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
  prompt = "你是 {}. 你的计划是: {}. 你目前在 {}. 当前时间为 {}:00. 你可以观察到以下内容: {}. 代入角色以你的角度用一句话概括这些记忆。".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  res = generate(prompt_meta.format(prompt))
  compressed_memories[name] = '[在 {}:00 的回忆: {}]'.format(str(global_time), res)
  compressed_memories_all[name].append(compressed_memories[name])

-the prompt:
泰瑞尔·血痕的记忆提到了杀死所有人,但这只是一则不相关的事件,对了解镇子和每个人的故事并没有任何帮助,因此将其评分为1分。
-the out
-the prompt:
这些信息是关于达兰·艾德玛斯保护苹果园和迎接新挑战的计划。根据描述,他对保护苹果园和迎接新挑战非常关心,但目前对保护苹果园的具体计划只有到8点的记忆,因此无法判断他的具体关心程度。同时,他与哈宾·韦斯特和盖蕾尔修女的交流,以及建立矿工交易所成为城镇最接近治理权力的事物都对其计划产生影响,但对其具体关心程度没有明确说明。
-the out
-the prompt:
莱宁·格雷温德在芬达林镇广场,关注建立矿工交易所成为最接近治理权力的事物、为镇上的人提供最好的服务、了解他们的需求和目标,但兴趣有限,对泰瑞尔·血痕和芬达林镇广场暴力事件的信息则知之甚少。
-the out
-the prompt:
哈莉亚的目标是建立矿工交易所成为城镇最接近治理权力的事物。
-the out
-the prompt:
奎琳·奥德利芙在芬达林镇广场,目前时间为8:00,对探索镇上的人和事情、了解其历史和未来发展感兴趣,但谋杀案影响其心情,评分为5分。
-the out
-the prompt:
盖蕾尔修女在芬达林镇广场,对保护苹果园和与村民互动、了解所有人目标非常关心,评分为5分；对了解所有人目标、哈宾·韦斯特和芬达林镇未来非常关心,评分为5分；对泰瑞尔·血痕杀死了所有人这一事件非常关心,评分为3分。
-the out
-the prompt:
这段记忆表明,哈宾·韦斯特对于查看芬达林镇的未来并不关心,因为他已经预见到了结局。但他仍然需要了解达兰·艾德玛斯和其他人的情况,以确保小镇的未来。
-the out
-the prompt:
泰瑞尔·血痕在芬达林镇广场,计划杀死所有人,但非常担心这样做会伤害其他人,正在努力与镇上的人交流并建立保护苹果园的计划。
-the out
-the prompt:
妮莉·星匠在芬达林镇广场,计划杀死红袍暴徒,但同时也保护苹果园和与村民互动。关心程度为5分,现在将分为1到5分。
-the out


In [14]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    prompt = "你是 {}. 你的计划是: {}. 你目前在 {}. 当前时间为 {}:00. 你有以下记忆: {}. 请给出一个1到5之间的评分,用于表示你在接下来的一个小时内有多大可能会去 {}".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name], 
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = '[在 {}:00 的回忆: {}]'.format(str(global_time), '然后你移动到了 {}.'.format(place_ratings_sorted[0][0]))
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


# 程序汇总

In [18]:
for repeats in range(5):
  global_time += 1
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      prompt = "你是 {}. 你的计划是: {}. 你目前在 {} 其描述如下: {}. 你的记忆是: {}. 当前时间为 {}:00. 这个区域里有以下人员: {}. 你可以其中某些人进行互动。".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      people_description = []
      for i in people:
        people_description.append(i+': '+town_people[i])
      prompt += ' 你知道以下人员的信息: ' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "接下来的一个小时你要做什么?最多用20个字来描述。"
      action_prompts[name] = prompt
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(prompt_meta.format(action_prompts[name]))
    # Now clean the action
    prompt = """
    将以下段落转换为第一人称过去式:
    "{}"
    """.format(action_results[name])
    action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  action_emojis = {}
  for name in town_people.keys():
    prompt = """
    将下面的段落转换为tuple (Action, Object):
    "{}"
    """.format(action_results[name])
    action_emojis[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print('    - Emoji 表示:', name, locations[name], global_time, action_emojis[name])
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      for name_two in people:
        memories[name].append('[时间: {}. 人物: {}. 记忆: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt = "你是 {}. 你的计划是: {}. 你的记忆是: {}. 你目前在 {}. 当前时间为 {}:00. 这个区域里有以下人员: {}. 将你对于这件事关心的程度评为1到5分。".format(name, plans[name], '\n'.join(compressed_memories_all[name][-5:]), locations[name], str(global_time), memory)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((res, rating))

  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(
          memory_ratings[name], 
          key=lambda x: x[1]
      )[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
    prompt = "你是 {}. 你的计划是: {}. 你目前在 {}. 当前时间为 {}:00. 你可以观察到以下内容: {}. 代入角色以你的角度用一句话概括这些记忆。".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
    res = generate(prompt_meta.format(prompt))
    compressed_memories[name] = '[在 {}:00 的回忆: {}]'.format(str(global_time), res)
    compressed_memories_all[name].append(compressed_memories[name])

  place_ratings = {}

  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt = "你是 {}. 你的计划是: {}. 你目前在 {}. 当前时间为 {}:00. 你有以下记忆: {}. 请给出一个1到5之间的评分,用于表示你在接下来的一个小时内有多大可能会去 {}。".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(
        place_ratings[name], 
        key=lambda x: x[1] )[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = '[在 {}:00 的回忆: {}]'.format(str(global_time), '然后你移动到了 {}.'.format(place_ratings_sorted[0][0]))
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]


/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Toblen Stonehill Edermath Orchard 16 I explored the area to find a new book that will help me improve my research on the trade secrets of the Phandalin Trade Company. I may ask Terrill Bloodscar for help, as Terrill is a ruffian I know.
Daran Edermath The Sleeping Giant 16 Daran Edermath talked to Halia Thornton and Sister Garaele.
Linene Graywind Phandalin Town Square 16 Linene created a marketing campaign to increase retail sales at the trading post.
Halia Thornton The Sleeping Giant 16 I got the conversation started. I discussed the upcoming events and had a fun time.
Qelline Alderleaf Tresendar Manor 16 Quilline went to the Shrine of Luck, focused on Daran Edermath, visited his family. She was interested in learning about their familys history and culture, including the traditions of their faith and other aspects of the history or magic in Phandalin. After visiting his family, Quilline moved to Tresendar Manor, with the goal of talking to Valerie and learning about the locals. Ther

KeyboardInterrupt: ignored

In [22]:
# 导出的目录
import json
with open('D:/generatedtext/plans.json', 'w',encoding='utf-8') as f:
    json.dump(plans, f, ensure_ascii=False)
with open('D:/generatedtext/location.json', 'w',encoding='utf-8') as f:
    json.dump(location, f, ensure_ascii=False)
with open('D:/generatedtext/memories.json', 'w',encoding='utf-8') as f:
    json.dump(memories,f, ensure_ascii=False)
with open('D:/generatedtext/compressed_memories_all.json', 'w',encoding='utf-8') as f:
    json.dump(compressed_memories_all, f, ensure_ascii=False)
with open('D:/generatedtext/memory_ratings.json', 'w',encoding='utf-8') as f:
    json.dump(people, f, ensure_ascii=False)